## 1. Create endpoint config
In this section we will describe how we can create a model endpoint. If you already have endpoint , please go to last cell for invocation, give the correct end point name and start inferences.

For generating sagemaker end point we will need to generate endpoint config. So generating endpoint config first

In [1]:
%store -r # you can read model from another notebook via this method 
print(v_model_name)

ds-mlops-linear-learner


In [2]:
import sys
import math
import boto3
import sagemaker
from time import gmtime, strftime
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
runtime_client = boto3.client("runtime.sagemaker", region_name=region)
client = boto3.client("sagemaker", region_name=region)


In [4]:
# Instance config for endpoint
endpoint_config_name = 'ds-mlops-linear-learner-endpoint-configV1'
v_model_name = "wipcoe-pricing-ml-training-pipeline-lr-3196968970"
Instance_Type = "ml.m5.large"
VolumeSizeIn_GB = 20
Instance_Count = 1
endpoint_name = 'wipcoe-pricing-ml-training-pipeline-lr-endpoint' # name of model endpoint

## 2. Delete End point Config
Incase if you want to delete endpoint config  and recreate it we can use below code snippet

In [7]:
response = client.delete_endpoint_config(
     EndpointConfigName=endpoint_config_name
 )

In [11]:
# Creating endpoint config
print(endpoint_config_name)
create_endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": Instance_Type,
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": v_model_name,
            "VariantName": "AllTraffic",
        }
    ],
    DataCaptureConfig={
        'EnableCapture': True,
        'InitialSamplingPercentage': 100,
        'DestinationS3Uri': 's3://ds-mlops-s3/data/capture/',
        'CaptureOptions': [
            {
                'CaptureMode': 'Input'
            },
             {
                'CaptureMode': 'Output'
            },
        ],
        'CaptureContentTypeHeader': {
            'CsvContentTypes': [
                'text/csv',
            ],
            'JsonContentTypes': [
                'application/json',
            ]
        }        
    }
)

print(f"Endpoint Config Arn: {create_endpoint_config_response['EndpointConfigArn']}")

ds-mlops-linear-learner-endpoint-configV1


ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: Cannot create already existing endpoint configuration "arn:aws:sagemaker:us-east-1:525102048888:endpoint-config/ds-mlops-linear-learner-endpoint-configv1".

**Note :** We will use able endpoint config name to cretae model endpoint

## 3. Delete endpoint

Incase if you want to delete endpoint  and recreate it we can use below code snippet

In [ ]:
client.delete_endpoint(EndpointName=endpoint_name)

## 4.Create Endpoint

In this notebook we create endpoint using endpoint which we have created

In [12]:
# Importing required library
import sys
import math
import boto3
import sagemaker
import time    

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
runtime_client = boto3.client("runtime.sagemaker", region_name=region)
client = boto3.client("sagemaker", region_name=region)


In [13]:
# Creating model endpoint
print(endpoint_name)
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

resp = client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
while status == "Creating":
    print(f"Status: {status}")
    time.sleep(60)
    resp = client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]

print(f"Arn: {resp['EndpointArn']}")
print(f"Status: {status}")

ds-mlops-linear-learner-endpoint
arn:aws:sagemaker:us-east-1:525102048888:endpoint/ds-mlops-linear-learner-endpoint
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Arn: arn:aws:sagemaker:us-east-1:525102048888:endpoint/ds-mlops-linear-learner-endpoint
Status: InService


## 5. Inference

In this section we will validate scoring logics. In below code snippet we are scoring with 14 predictor values and finding the result

In [10]:
print(lrbody)

{"instances": [{"features": ["2018", "5", "147", "2", "4", "2", "2000", "0", "2", "3", "8", "6"]}]}


In [11]:
import json
payload={'year': 2018, 'manufacturer': 5, 'model': '147', 'condition': '2', 'cylinders': '4', 'fuel': '2', 'odometer': '2000', 'title_status': 0, 'transmission': '2', 'drive': '3', 'type': '8', 'paint_color': '6'}
scoring_vector = [payload['year'],payload['manufacturer'],payload['model'],payload['condition'],payload['cylinders'],payload['fuel'],payload['odometer'],payload['title_status'],payload['transmission'],payload['drive'],payload['type'],payload['paint_color']]
lrbodyvector = ([str(item) for item in scoring_vector])
lrbody=json.dumps({
    "instances":[
    {"features":lrbodyvector 
    }]})

client = boto3.client('sagemaker-runtime')
response_model_linear_learner = client.invoke_endpoint(EndpointName=endpoint_name,#'wipcoe-pricing-ml-training-pipeline-lr-endpoint',
                               ContentType='application/json',
                               Body=lrbody)
print(response_model_linear_learner)
result_linear_learner = json.loads(str(response_model_linear_learner["Body"].read().decode('utf8')))
print(result_linear_learner)
result_linear_learner = float(result_linear_learner["predictions"][0]["score"])
print(result_linear_learner)

{'ResponseMetadata': {'RequestId': '78e7da1e-fe70-46c1-aac7-bdb93d27e026', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '78e7da1e-fe70-46c1-aac7-bdb93d27e026', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Wed, 11 Aug 2021 08:37:40 GMT', 'content-type': 'application/json', 'content-length': '42'}, 'RetryAttempts': 0}, 'ContentType': 'application/json', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7f47180d0438>}
{'predictions': [{'score': 8314.8046875}]}
8314.8046875


**Note:** Similarly you can change the test vector and can see how endpoint is working